In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [20]:
files = glob('forums/*.json')
len(files)

12

In [21]:
forums = []
for f in files:
    with open(f) as fopen:
        forums.extend(json.load(fopen))
        
forums = sorted(list(set(forums)))
len(forums)

9784

In [22]:
forums[0]

'https://forums.salary.sg/bitcoin-cryptocurrency-talk/10000-how-buy-bitcoin-singapore.html'

In [31]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 10.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    div = soup.find_all('div')
    div = [str(div_) for div_ in div if 'post_message' in div_.get('id', '')]
    try:
        td = soup.find_all('td', {'class': 'vbmenu_control'})
        td = [td_.text for td_ in td if 'Page 1 of ' in td_.text][0]
        max_page = int(td.split('Page 1 of ')[1])
    except:
        max_page = -1
    return url, div, max_page

get_url(forums[0])

('https://forums.salary.sg/bitcoin-cryptocurrency-talk/10000-how-buy-bitcoin-singapore.html',
 ['<div id="post_message_103183">\nFor beginners, a quick way to get started is to use Coinbase, which is a US-based company that lets you to use your credit card to buy Bitcoins and Ethereum ethers. If you use our referral link below, you will get a bonus of S$13 worth of Bitcoin after spending S$134 there:<br/>\n<a href="https://www.coinbase.com/join/59eacfce21d34c00e2685e1f" target="_blank">https://www.coinbase.com/join/59eacfce21d34c00e2685e1f</a><br/>\n<br/>\nRead more here:<br/>\n<a href="http://www.salary.sg/2017/how-to-buy-bitcoin-in-singapore/" target="_blank">http://www.salary.sg/2017/how-to-buy...-in-singapore/</a>\n</div>',
  '<div id="post_message_103957">\nwhatever you invest in, you will need to have a good understanding of how it works before making a decision. Dont just jump into to hype because others is buying bitcoins.\n<br/>\n<br/>\n<div style="float:right">\n<script type=

In [27]:
!rm -rf threads
!mkdir threads

In [33]:
max_worker = 20

for i in tqdm(range(0, len(forums), max_worker)):
    
    filename = os.path.join('threads', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
        
    topics = [topic for topic in forums[i: i + max_worker]]
    
    data = []    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url): url for url in topics}

        for future in as_completed(futures):
            data.append(future.result())
                
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)
        
    try:
        del data
        del futures
    except:
        pass

100%|█████████████████████████████████████████| 490/490 [14:15<00:00,  1.75s/it]
